In [1]:
import gym

import env

env = gym.make('CREDITCARDFRAUD-v0')

obs = env.reset()

284314


C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'pandas.core.series.Series'>`
  logger.warn(


In [2]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

e:\conda\envs\yoloqt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
actions = []
rewards = []
count = 5

while True:
  action = env.action_space.sample()
  reward = env.step(action)
  actions.append(action)
  rewards.append(reward)
  """
  print(action)
  print(reward)
  """
  count -= 1
  if count==0:
    break


C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:133: UserWarning: WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'pandas.core.series.Series'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\gym\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [5]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils

In [ ]:
df = pd.read_csv('./data/train_dataset_balanced.csv')
df.head() 

In [6]:
"""
# Load the datasets
df_train = pd.read_csv('./data/train_dataset_balanced.csv')
df_test = pd.read_csv('./data/test_dataset_balanced.csv')

# Splitting the datasets into features and target variable
X_train = df_train.iloc[:, :-1].values
y_train = df_train['Class'].values
X_test = df_test.iloc[:,: -1].values
y_test = df_test['Class'].values

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




X_train=X_train_scaled
X_test=X_test_scaled 
Y_train=y_train
Y_test=y_test

X_train, X_test_, Y_train, Y_test_ = train_test_split(X_train, Y_train, test_size=0.5, random_state=1)
"""

In [ ]:
"""
X = df.iloc[:, :-1].values 
y = df.iloc[:, -1].values

sc = StandardScaler()
X = sc.fit_transform(X)

"""

In [7]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        self.fc5 = nn.Linear(24, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))
        return x

In [8]:
model = DQN()
print(model)

DQN(
  (fc1): Linear(in_features=30, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=18, bias=True)
  (fc3): Linear(in_features=18, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=24, bias=True)
  (fc5): Linear(in_features=24, out_features=2, bias=True)
)


In [9]:
import random 
from collections import namedtuple, deque 

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  #replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate
UPDATE_EVERY = 4        # how often to update the network
EPSILON = 0.8           # probability of chosing on-policy action

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
class ReplayBuffer():
  def __init__(self, action_size, buffer_size, batch_size, seed):
    self.action_size = action_size
    self.memory = deque(maxlen=buffer_size)
    self.batch_size = batch_size
    self.experiences = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    self.seed = random.seed(seed)
  
  def add(self, state, action, reward, next_state, done):
    """
    print(f"Adding state shape: {state.shape}, next_state shape: {next_state.shape}")
    """
    experience = self.experiences(state, action, reward, next_state, done)
    self.memory.append(experience)
  """
  def sample(self):
     experiences = random.sample(self.memory, k=self.batch_size)
     for i, e in enumerate(experiences):
      if e is not None:
          print('1________')
          print(e)
          print('2________')
          print(f"Index {i}: {e.state.shape}")
          print('3________')
          print('')

     states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).double().to(device)
     actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).double().to(device)
     rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).double().to(device)
     next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).double().to(device)
     dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None])).double().to(device)
     return (states, actions, rewards, next_states, dones)
  """
  def sample(self):
    # 从经验池中随机采样一批数据
    experiences = random.sample(self.memory, k=self.batch_size)
    
    # 初始化空列表来存储每个数据的对应部分
    states = []
    actions = []
    rewards = []
    next_states = []
    dones = []
    
    # 遍历采样到的每个经验
    for i, e in enumerate(experiences):
        # 检查是否有 None 类型的经验
        if e is not None:
            # 将各部分数据添加到对应的列表中
            states.append(e.state)
            actions.append(e.action)
            rewards.append(e.reward)
            next_states.append(e.next_state)
            dones.append(e.done)
            """
            print('1________')
            print(e)
            print('2________')
            print(f"Index {i}: {e.state.shape}")
            print('3________')
            print('')
            """
    # 将列表转换为 NumPy 数组，并转换为 PyTorch 张量，并移动到指定设备
    states = torch.from_numpy(np.vstack(states)).double().to(device)
    actions = torch.from_numpy(np.vstack(actions)).double().to(device)
    rewards = torch.from_numpy(np.vstack(rewards)).double().to(device)
    next_states = torch.from_numpy(np.vstack(next_states)).double().to(device)
    dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).double().to(device)
    
    return (states, actions, rewards, next_states, dones)

    # experiences = random.sample(self.memory, k=BATCH_SIZE)

    # batch = self.experiences(*zip(experiences))

    # states = torch.cat(batch.state)
    # actions = torch.cat(batch.actions)
    # rewards = torch.cat(batch.reward)
    # next_states = torch.cat(batch.next_state)
    # dones = torch.cat(batch.done)
    #return random.sample(self.memory, BATCH_SIZE)

    
  
  def __len__(self):
      return len(self.memory)


In [11]:
class Agent():
  def __init__(self, action_size, seed):
    self.action_size = action_size
    self.seed = random.seed(seed)


    # Q - Network
    self.qnet_local = DQN().double().to(device)
    self.qnet_target = DQN().double().to(device)

    self.optimizer = optim.Adam(self.qnet_local.parameters(), lr=0.001)

    self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

    self.t_step = 0
    self.train_loss = []

  def step(self, state, action, reward, next_state, done):
    self.memory.add(state, action, reward, next_state, done)


    # learn every 4 timesteps
    self.t_step = (self.t_step+1)%64
    if self.t_step == 0:
      experience = self.memory.sample()
      #print('Experience sampled from memory : ', experience)
      self.learn(experience, GAMMA)


  def epsilon_greedy_action(self, state):
    state = state.to(device)
    self.qnet_local.eval()
    with torch.no_grad():
      action_values = self.qnet_local(state).max(1)[1]#.view(1, 1)
    self.qnet_local.train()

    if random.random() < 0.8:
      print('Predicted action based on QNetwork : ', action_values)
      return action_values.cpu()
    else:
      random_action = random.choices(np.arange(self.action_size), k=BATCH_SIZE)
      print('Chosing  random actions for the batch : ', random_action)
      return torch.DoubleTensor(random_action)
  
  def learn(self, experiences, gamma):
    #print('Started learning')
    states, actions, rewards, next_states, done = experiences#experiences[0].state, experiences[0].action, experiences[0].reward, experiences[0].next_state, experiences[0].done 
    criterion = torch.nn.BCELoss()
    self.qnet_local.train()
    self.qnet_target.eval()

    #predicted_targets = self.qnet_local(states)#.gather(1, actions)

    #print(next_states.view(1, 1))
    with torch.no_grad():
      labels_next = self.qnet_target(next_states).detach().max(1)[0].unsqueeze(1)
    
    #print('labels_next {}'.format(labels_next))
    
    labels = 0 + (gamma * labels_next)
    predicted_targets = self.qnet_local(states).gather(1, actions.long())

    #print("Predicted targets : {}, labels : {}".format(predicted_targets, labels))

    loss = criterion(predicted_targets, labels).to(device)
    print("===========================Training loss ============================")
    #print(loss.item())
    self.train_loss.append(loss.item())
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    print('Total training losses : ', self.train_loss)

    # perform soft update
    self.soft_update(self.qnet_local, self.qnet_target, TAU)
  
  def soft_update(self, local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau*local_param.data + (1-tau)*target_param.data)

In [12]:

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.5, random_state=42)

X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()

train = data_utils.TensorDataset(X_train, Y_train)
train_loader = data_utils.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
# check reward strategy once
# add probability to epsilon_greedy
import json
deep_agent = Agent(action_size=2, seed=0)
num_episodes = 1
max_t = 1000
state = 0
env.state_idx = 0

true_positive = []
true_negative = []

false_positive = []
false_negative = []

TPR = []
FPR = []

current_state = df.iloc[0, :-1].values

for i in range(1):  
  #print("==========================EPOCH {} COMPLETED===================".format(i))  

  print('Current state : ', i)
  score = 0
  for state_idx, data in enumerate(train_loader, 0):
    inputs, labels = data    
    action = deep_agent.epsilon_greedy_action(inputs)
    for a in action:
      next_state, reward, done, info = env.step(a)
      deep_agent.step(current_state, a, reward, next_state, done)
      current_state = next_state
      roc_info = info  # if info is already a dictionary
      
      
     

NameError: name 'df' is not defined

In [ ]:
import matplotlib.pyplot as plt

# 获取实际的训练损失数据长度
epochs = len(deep_agent.train_loss)

# 使用实际长度绘图
plt.plot(range(epochs), deep_agent.train_loss, color='orange', label='Training Loss')

# 添加图例
plt.legend(loc="upper right")

# 标注坐标轴
plt.xlabel('# Epochs')
plt.ylabel('Training Loss')

# 添加标题
plt.title('Training Loss Across Epochs')

# 显示图形
plt.show()


In [ ]:
X_test = torch.from_numpy(X_test)
Y_test = torch.from_numpy(Y_test).double()

In [ ]:
test = data_utils.TensorDataset(X_test, Y_test)
test_loader = data_utils.DataLoader(test, batch_size=64, shuffle=True)

In [ ]:
import torch
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# 初始化用于存储预测和标签的列表
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)  # 确保数据在模型所在的设备上
        predicted = deep_agent.epsilon_greedy_action(inputs)
        y_pred.extend(predicted.cpu().numpy())  # 保存预测结果
        y_true.extend(labels.cpu().numpy())  # 保存真实标签

# 计算混淆矩阵
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# 使用 classification_report 输出分类性能结果
report = classification_report(y_true, y_pred, target_names=['not_fraud', 'fraud'])
print(report)
